In [7]:
import torch
from torch.nn import Module
from torch.autograd import Variable
import torchvision
from torchvision.datasets import MNIST
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torch.nn.functional as F

In [2]:
# nn.Linear?

In [3]:
train_mnist_transforms = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (1.0,))])
train_mnist_dataset = MNIST(root='~/Data/pytorch/', train=True, transform=train_mnist_transforms)

In [4]:
batch_size=1

In [5]:
train_mnist_data_loader = torch.utils.data.DataLoader(dataset=train_mnist_dataset,batch_size=batch_size,shuffle=True)

In [6]:
# for epoch in xrange(1):
#     for batch_id, (x, y) in enumerate(train_mnist_data_loader):
#         x, y = Variable(x), Variable(y)
#         print('x:', x.data.shape)
#         print('y:', y.data.shape)
#         break

In [10]:
class Conv_Net(Module):
    def __init__(self):
        super(Conv_Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, (5,5), padding=2)
        self.conv2 = nn.Conv2d(6, 16, (5,5))
        self.fc1   = nn.Linear(16*5*5, 120)
        self.fc2   = nn.Linear(120, 84)
        self.fc3   = nn.Linear(84, 10)
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2,2))
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [11]:
model = Conv_Net()

In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-4)

In [ ]:
for epoch in range(10):
    loss_epoch = 0
    loss_avg_epoch = 0
    data_count = 0
    right_count = 0
    for i, (x, y) in enumerate(train_mnist_data_loader):
        data_count = i
#         print(i)
        x, y = Variable(x), Variable(y)

        # 训练优化参数
        optimizer.zero_grad()

        outputs = model(x)
        loss = criterion(outputs, y)
        # print('loss:', loss)
        loss_numpy = loss.data.numpy()
        loss_epoch += loss_numpy
        loss.backward()

        optimizer.step()
        outputs_np = outputs.data.numpy()
        pred = np.argmax(outputs_np, axis=1)
        y_np = y.data.numpy()
        right_count += (pred==y_np)
#         print('outputs_numpy:', outputs_numpy)
#         print('pred:', pred)
#         print('y_np:', y_np)
#         break

    print('acc:', right_count*1.0/data_count)
    # 输出一个周期后的loss
    loss_avg_epoch = loss_epoch / (data_count * batch_size * 1.0)
    print('epoch:', epoch)
    print('loss_avg_epoch:', loss_avg_epoch)

('acc:', array([ 0.94063234]))
('epoch:', 0)
('loss_avg_epoch:', array([ 0.18523914], dtype=float32))
('acc:', array([ 0.98143302]))
('epoch:', 1)
('loss_avg_epoch:', array([ 0.06200045], dtype=float32))
('acc:', array([ 0.98511642]))
('epoch:', 2)
('loss_avg_epoch:', array([ 0.04887283], dtype=float32))
